In [ ]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import csv

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("datamunge/sign-language-mnist")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/datamunge/sign-language-mnist/versions/1


In [ ]:
# Construct file paths
path_sign_mnist_train = f"{path}/sign_mnist_train.csv"
path_sign_mnist_test = f"{path}/sign_mnist_test.csv"

def get_data(filename):
    with open(filename, 'r') as training_file:
        csv_reader = csv.reader(training_file, delimiter=',')
        first_line = True
        temp_images = []
        temp_labels = []
        for row in csv_reader:
            if first_line:
                # Skip header line
                first_line = False
            else:
                # Append label (first value)
                temp_labels.append(int(row[0]))
                # Append image data as a 28x28 array
                image_data = np.array(row[1:785], dtype=float)
                image_data_as_array = image_data.reshape(28, 28)
                temp_images.append(image_data_as_array)

        # Convert lists to numpy arrays
        images = np.array(temp_images)
        labels = np.array(temp_labels)
    return images, labels

# Load data
training_images, training_labels = get_data(path_sign_mnist_train)
testing_images, testing_labels = get_data(path_sign_mnist_test)

# Print shapes
print(training_images.shape)
print(training_labels.shape)
print(testing_images.shape)
print(testing_labels.shape)


(27455, 28, 28)
(27455,)
(7172, 28, 28)
(7172,)


In [ ]:
training_images = training_images.reshape((training_images.shape[0], 28 * 28))
testing_images = testing_images.reshape((testing_images.shape[0], 28 * 28))

#normalizing the pixel values
training_images = training_images / 255.0
testing_images = testing_images / 255.0

print(training_images.shape)
print(testing_images.shape)


(27455, 784)
(7172, 784)


In [ ]:
#from sklearn.preprocessing import OneHotEncoder

# Convert labels to one-hot encoding
#encoder = OneHotEncoder(sparse_output=False)  # Set sparse=False to return a dense matrix

#training_labels = encoder.fit_transform(training_labels.reshape(-1, 1))
#testing_labels = encoder.transform(testing_labels.reshape(-1, 1))

#print(testing_labels)

In [ ]:
#coNVERT THE data into Tensors
training_images = torch.tensor(training_images, dtype=torch.float32)
testing_images = torch.tensor(testing_images, dtype=torch.float32)
training_labels = torch.tensor(training_labels, dtype=torch.float32)
testing_labels = torch.tensor(testing_labels, dtype=torch.float32)

print(training_labels)
testing_labels = testing_labels.long()
training_labels = training_labels.long()
print(training_labels)


tensor([ 3.,  6.,  2.,  ..., 18., 17., 23.])
tensor([ 3,  6,  2,  ..., 18, 17, 23])


In [ ]:
from torch import nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # Input layer (28x28 = 784 features)
        self.dropout1 = nn.Dropout(p=0.2)   # Dropout after first layer (20% drop)
        self.fc2 = nn.Linear(128, 64)       # Hidden layer
        self.dropout2 = nn.Dropout(p=0.2)   # Dropout after second layer (20% drop)
        self.fc3 = nn.Linear(64, 26)        # Output layer (26 classes for sign language)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # ReLU activation function after the first layer
        x = self.dropout1(x)         # Apply Dropout after first layer
        x = torch.relu(self.fc2(x))  # ReLU activation function after the second layer
        x = self.dropout2(x)         # Apply Dropout after second layer
        x = self.fc3(x)              # Output layer
        return x


In [117]:
import torch.optim as optim

model = NeuralNetwork()
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 100

for epoch in range(epochs):

  model.train()

  output = model(training_images)

  loss_val = criterion(output, training_labels)
 # l2_norm = sum(p.pow(2).sum() for p in model.parameters())
 # loss_val += 0.01 * l2_norm

  loss_val.backward()

  optimizer.step()

  optimizer.zero_grad()

  if epoch % 10 == 0:
    print(f"Epoch {epoch}, Loss: {loss_val.item()}")



Epoch 0, Loss: 3.2601711750030518
Epoch 10, Loss: 3.1455743312835693
Epoch 20, Loss: 2.9648382663726807
Epoch 30, Loss: 2.7399308681488037
Epoch 40, Loss: 2.49904465675354
Epoch 50, Loss: 2.276210069656372
Epoch 60, Loss: 2.0894198417663574
Epoch 70, Loss: 1.9451874494552612
Epoch 80, Loss: 1.827473521232605
Epoch 90, Loss: 1.71791410446167


In [ ]:
model.eval()  # Set the model to evaluation mode
with torch.no_grad():  # Disable gradient calculation for inference
    testing_images = testing_images.view(testing_images.size(0), -1)
    test_outputs = model(testing_images)
    _, predicted = torch.max(test_outputs, 1)

    accuracy = (predicted == testing_labels).float().mean()

    print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")


Test Accuracy: 53.78%
